In [1]:
import os
import cv2
import numpy as np
import random


Function for copy paste

In [ ]:
def paste_objects(source_image1, mask_images1, source_image2, mask_images2, background_image, name_info, output_dir):
    # Create output directories if they don't exist
    augmented_image_dir = os.path.join(output_dir, "augmented_images")
    augmented_mask_dir = os.path.join(output_dir, "augmented_masks")
    os.makedirs(augmented_image_dir, exist_ok=True)
    os.makedirs(augmented_mask_dir, exist_ok=True)

    # Convert the mask images to binary masks
    binary_masks1 = []
    for mask_image1 in mask_images1:
        _, binary_mask1 = cv2.threshold(mask_image1, 127, 255, cv2.THRESH_BINARY)
        binary_masks1.append(binary_mask1)

    binary_masks2 = []
    for mask_image2 in mask_images2:
        _, binary_mask2 = cv2.threshold(mask_image2, 127, 255, cv2.THRESH_BINARY)
        binary_masks2.append(binary_mask2)

    # Find contours in the binary masks
    contours1_list = []
    for binary_mask1 in binary_masks1:
        contours1, _ = cv2.findContours(binary_mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours1_list.append(contours1)
    
    contours2_list = []
    for binary_mask2 in binary_masks2:
        contours2, _ = cv2.findContours(binary_mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours2_list.append(contours2)
        
    # Shuffle the contours to get a random order
    shuffled_contours1 = []
    for contours1 in contours1_list:
        shuffled_contours1.append(random.sample(list(contours1), 1))
    
    shuffled_contours2 = []
    for contours2 in contours2_list:
        shuffled_contours2.append(random.sample(list(contours2), 1))
    
    augmented_masks1 = []
    augmented_masks2 = []
#     log = []
    
    for i, selected_contours1 in enumerate(shuffled_contours1):
        for selected_contour1 in selected_contours1:
            # Get the minimum bounding rectangle around the contour
            x1, y1, w1, h1 = cv2.boundingRect(selected_contour1)
            
            # Extract the subset of objects from the source image and mask
            object_subset1 = source_image1[y1:y1+h1, x1:x1+w1]
            mask_subset1 = mask_images1[i][y1:y1+h1, x1:x1+w1]
            
            # Create a mask of the same shape as the augmented image
            augmented_mask1 = np.zeros_like(background_image, dtype=np.uint8)
            
            # Convert the object mask to 3-channel mask
            mask_subset1 = cv2.cvtColor(mask_subset1, cv2.COLOR_GRAY2BGR)
            
            # Paste the object mask onto the augmented mask at the appropriate position
            paste_x1 = random.randint(0, background_image.shape[1] - w1)
            paste_y1 = random.randint(0, background_image.shape[0] - h1)
            augmented_mask1[paste_y1:paste_y1+h1, paste_x1:paste_x1+w1] = mask_subset1
            
            # Paste the object onto the background image at the random position
            background_image[paste_y1:paste_y1+h1, paste_x1:paste_x1+w1] = object_subset1
            
            # Adjust the ground-truth annotations and create the new contour
            adjusted_contour1 = selected_contour1.copy()
            adjusted_contour1[:, 0, 0] -= x1
            adjusted_contour1[:, 0, 1] -= y1
            adjusted_contour1[:, 0, 0] += paste_x1
            adjusted_contour1[:, 0, 1] += paste_y1
            augmented_masks1.append((augmented_mask1, adjusted_contour1))
            
#             log.append((i, 1))  # Log the indices of the images used for augmentation
            
    for i, selected_contours2 in enumerate(shuffled_contours2):
        for selected_contour2 in selected_contours2:
            # Get the minimum bounding rectangle around the contour
            x2, y2, w2, h2 = cv2.boundingRect(selected_contour2)
            
            # Extract the subset of objects from the source image and mask
            object_subset2 = source_image2[y2:y2+h2, x2:x2+w2]
            mask_subset2 = mask_images2[i][y2:y2+h2, x2:x2+w2]
            
            # Create a mask of the same shape as the augmented image
            augmented_mask2 = np.zeros_like(background_image, dtype=np.uint8)
            
            # Convert the object mask to 3-channel mask
            mask_subset2 = cv2.cvtColor(mask_subset2, cv2.COLOR_GRAY2BGR)
            
            # Paste the object mask onto the augmented mask at the appropriate position
            paste_x2 = random.randint(0, background_image.shape[1] - w2)
            paste_y2 = random.randint(0, background_image.shape[0] - h2)
            augmented_mask2[paste_y2:paste_y2+h2, paste_x2:paste_x2+w2] = mask_subset2
            
            # Paste the object onto the background image at the random position
            background_image[paste_y2:paste_y2+h2, paste_x2:paste_x2+w2] = object_subset2
            
            # Adjust the ground-truth annotations and create the new contour
            adjusted_contour2 = selected_contour2.copy()
            adjusted_contour2[:, 0, 0] -= x2
            adjusted_contour2[:, 0, 1] -= y2
            adjusted_contour2[:, 0, 0] += paste_x2
            adjusted_contour2[:, 0, 1] += paste_y2
            augmented_masks2.append((augmented_mask2, adjusted_contour2))
            
#             log.append((i, 2))  # Log the indices of the images used for augmentation
    
    # Generate the augmented image name
    source_image1_name = name_info[0]
    source_image2_name = name_info[1]
    augmented_image_name = f"{source_image1_name}and{source_image2_name}"
    augmented_image_path = os.path.join(augmented_image_dir, augmented_image_name + ".jpg")

    # Save the augmented image
    cv2.imwrite(augmented_image_path, background_image)

    # Generate and save the corresponding mask images
    for i, (augmented_mask, contour) in enumerate(augmented_masks1):
        mask_name = f"{augmented_image_name}_{name_info[i+2]}_{i}.jpg"
        mask_path = os.path.join(augmented_mask_dir, mask_name)
        cv2.imwrite(mask_path, augmented_mask)
        
    for i, (augmented_mask, contour) in enumerate(augmented_masks2):
        mask_name = f"{augmented_image_name}_{name_info[i+4]}_{i}.jpg"
        mask_path = os.path.join(augmented_mask_dir, mask_name)
        cv2.imwrite(mask_path, augmented_mask)
        
    return augmented_image_path, augmented_masks1, augmented_masks2

Calling function and giving inputs

In [ ]:
# Specify the directories for source images and masks
source_image_dir = './Car-Parts-Segmentation-master/trainingset/JPEGImages/'
mask_image_dir = './Car_parts_masked_segbinary/'

# Load the background image
background_image = cv2.imread('./training/ADE_train_00000974.jpg')

# Get the list of files in the directories
source_image_files = os.listdir(source_image_dir)
mask_image_files = os.listdir(mask_image_dir)

# List to store naming data

name_info = []

# Randomly choose two source images
chosen_source_images = random.sample(source_image_files, 2)

# Select two random masks for each chosen source image
chosen_masks = []
for source_image in chosen_source_images:
    image_name = os.path.splitext(source_image)[0]
    name_info.append(image_name)
    matching_masks = [mask for mask in mask_image_files if mask.startswith(image_name)]
    chosen_masks.extend(random.sample(matching_masks, 2))

for i in range(0,4):
    name_info.append(os.path.splitext(chosen_masks[i])[0])

# Load the chosen source images and masks
source_image1 = cv2.imread(source_image_dir + chosen_source_images[0])
mask_image1_1 = cv2.imread(mask_image_dir + chosen_masks[0], 0)
mask_image1_2 = cv2.imread(mask_image_dir + chosen_masks[1], 0)
mask_images1 = [mask_image1_1, mask_image1_2]

source_image2 = cv2.imread(source_image_dir + chosen_source_images[1])
mask_image2_1 = cv2.imread(mask_image_dir + chosen_masks[2], 0)
mask_image2_2 = cv2.imread(mask_image_dir + chosen_masks[3], 0)
mask_images2 = [mask_image2_1, mask_image2_2]

output_dir = "./test_folder/"

# Apply the paste operation
augmented_image, augmented_masks1, augmented_masks2 = paste_objects(source_image1, mask_images1, source_image2, mask_images2, background_image, name_info, output_dir)



Function to choose a file randomly from a directory

In [ ]:
def choose_random_image(directory):
    # Get a list of all image files in the directory
    image_files = [file for file in os.listdir(directory) if file.endswith(('.jpg', '.jpeg', '.png', '.gif'))]

    if not image_files:
        print("No image files found in the directory.")
        return None

    # Choose a random image file from the list
    random_image = random.choice(image_files)

    # Return the path to the random image file
    return os.path.join(directory, random_image)